# Router RAG Agent Basic


Router RAG Agent는 사용자의 질문을 분석하여 적절한 RAG(Retrieval-Augmented Generation) 데이터베이스로 라우팅하는 시스템이다. 예를 들어 기술 문서, 제품 정보, FAQ 등 여러 도메인의 문서가 있을 때, 질문의 특성에 따라 가장 관련성 높은 데이터베이스를 선택하여 검색한다.

기본 작동 원리는 다음과 같다:
1. 사용자 질문을 입력받는다
2. LLM이 질문의 카테고리를 분류한다
3. 분류된 카테고리에 해당하는 RAG 데이터베이스에서 검색한다
4. 검색된 정보를 바탕으로 최종 답변을 생성한다

In [ ]:
# 필요한 라이브러리 설치
%pip install -q openai

In [1]:
from dotenv import load_dotenv
import os

# .env 파일에 key를 저장하고 환경 변수를 로드한다
!echo "OPENAI_API_KEY=<sk-proj로 시작하는 OpenAI API Key를 복사 후 붙혀넣기>" >> .env
load_dotenv()

True

In [2]:
from openai import OpenAI
import json

# OpenAI 클라이언트 초기화
client = OpenAI()

In [3]:
# 각 카테고리별 RAG 데이터베이스 (간단한 예시)
rag_databases = {
    "기술문서": [
        "Python은 고수준 프로그래밍 언어이다.",
        "REST API는 HTTP 프로토콜을 사용한다.",
        "Docker는 컨테이너 기반 가상화 플랫폼이다."
    ],
    "제품정보": [
        "스마트폰 X는 6.5인치 디스플레이를 탑재했다.",
        "노트북 Y는 16GB RAM을 기본 제공한다.",
        "태블릿 Z는 방수 기능이 있다."
    ],
    "FAQ": [
        "배송은 주문 후 2-3일 소요된다.",
        "환불은 구매 후 7일 이내 가능하다.",
        "고객센터는 평일 9시부터 6시까지 운영한다."
    ]
}

In [4]:
def classify_query(query):
    """질문을 분석하여 적절한 카테고리로 분류한다"""
    
    prompt = f"""
    다음 질문을 분석하여 가장 적절한 카테고리를 선택하라.
    
    카테고리: 기술문서, 제품정보, FAQ
    
    질문: {query}
    
    카테고리 하나만 답변하라.
    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    
    category = response.choices[0].message.content.strip()
    return category

In [5]:
def retrieve_from_rag(category, query):
    """선택된 카테고리의 RAG 데이터베이스에서 관련 정보를 검색한다"""
    
    # 해당 카테고리의 모든 문서를 가져온다 (실제로는 벡터 유사도 검색 사용)
    documents = rag_databases.get(category, [])
    
    # 간단한 키워드 매칭 (실제로는 임베딩 기반 검색)
    relevant_docs = documents
    
    return relevant_docs

In [6]:
def generate_answer(query, context):
    """검색된 문서를 바탕으로 최종 답변을 생성한다"""
    
    prompt = f"""
    다음 정보를 바탕으로 질문에 답변하라.
    
    정보:
    {context}
    
    질문: {query}
    
    답변:
    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    
    answer = response.choices[0].message.content
    return answer

In [7]:
def router_rag_agent(query):
    """Router RAG Agent의 전체 파이프라인을 실행한다"""
    
    print(f"질문: {query}\n")
    
    # 1단계: 카테고리 분류
    category = classify_query(query)
    print(f"분류된 카테고리: {category}\n")
    
    # 2단계: RAG 검색
    documents = retrieve_from_rag(category, query)
    context = "\n".join(documents)
    print(f"검색된 문서:\n{context}\n")
    
    # 3단계: 답변 생성
    answer = generate_answer(query, context)
    print(f"최종 답변:\n{answer}")
    
    return answer

In [8]:
# 테스트 실행
test_queries = [
    "Python은 어떤 언어인가?",
    "노트북 Y의 사양이 궁금하다",
    "배송은 얼마나 걸리나?"
]

for query in test_queries:
    print("=" * 30)
    router_rag_agent(query)
    print("=" * 30)
    print()

질문: Python은 어떤 언어인가?

분류된 카테고리: 기술문서

검색된 문서:
Python은 고수준 프로그래밍 언어이다.
REST API는 HTTP 프로토콜을 사용한다.
Docker는 컨테이너 기반 가상화 플랫폼이다.

최종 답변:
Python은 고수준 프로그래밍 언어입니다.

질문: 노트북 Y의 사양이 궁금하다

분류된 카테고리: 제품정보

검색된 문서:
스마트폰 X는 6.5인치 디스플레이를 탑재했다.
노트북 Y는 16GB RAM을 기본 제공한다.
태블릿 Z는 방수 기능이 있다.

최종 답변:
노트북 Y는 16GB RAM을 기본으로 제공하며, 그 외의 사양에 대한 정보는 제공되지 않았습니다. 추가적인 사양, 예를 들어 프로세서, 저장 용량, 그래픽 카드 등의 정보는 확인할 수 없습니다. 더 궁금한 사항이 있으시면 말씀해 주세요!

질문: 배송은 얼마나 걸리나?

분류된 카테고리: FAQ

검색된 문서:
배송은 주문 후 2-3일 소요된다.
환불은 구매 후 7일 이내 가능하다.
고객센터는 평일 9시부터 6시까지 운영한다.

최종 답변:
배송은 주문 후 2-3일 소요됩니다.

